In [1]:
from sqlalchemy import create_engine, exc
import pandas as pd

In [2]:
usuario = 'root'
senha = 'root'
host = 'localhost'
portorigem = 3308
portdestino = 3306
bancoorigem = 'vendas'
bancodestino = 'vendas_dimensional'

In [3]:
try:
    engineori = create_engine(f"mysql+pymysql://{usuario}:{senha}@{host}:{portorigem}/{bancoorigem}")
    with engineori.connect() as connori:
        print(f"Conexão com o banco '{bancoorigem}' estabelecida com sucesso.")
except exc.SQLAlchemyError as e:
    print(f"Erro ao conectar ao banco '{bancoorigem}': {e}")
    exit()

Conexão com o banco 'vendas' estabelecida com sucesso.


In [4]:
try:
    enginedes = create_engine(f"mysql+pymysql://{usuario}:{senha}@{host}:{portdestino}/{bancodestino}")
    with enginedes.connect() as conndes:
        print(f"Conexão com o banco '{bancodestino}' estabelecida com sucesso.")
except exc.SQLAlchemyError as e:
    print(f"Erro ao conectar ao banco '{bancodestino}': {e}")
    exit()

Conexão com o banco 'vendas_dimensional' estabelecida com sucesso.


In [5]:
sql = f"""
     select cl.id,cl.nome,cl.sexo,cl.profissao,cl.nacionalidade,pa2.pais as nac,cl.email,cl.nascimento,
            cl.cadastro, cl.endereco,cl.bairro,cl.cidade,cl.estado,cl.cep,pa1.pais
     from clientes cl
     inner join paises pa1 on cl.pais = pa1.sigla
     inner join paises pa2 on cl.nacionalidade = pa2.sigla
     """
clientes = pd.read_sql(sql,engineori)
clientes = clientes.reset_index()
clientes['provedor'] = clientes['email'].apply(lambda x: x.split('@')[1])
clientes['enderecocompleto'] = clientes.apply(lambda row: 
    f"{row['endereco']}, {row['bairro']}, {row['cidade']} - {row['estado']}, CEP: {row['cep']},{row['pais']}",
    axis=1
)
clientes['sexodescricao'] = clientes['sexo'].map({
    'f': 'Feminino',
    'F': 'Feminino',
    'm': 'Masculino',
    'M': 'Masculino',
    'o': 'Outros',
    'O': 'Outros'
}).fillna('Não Informado')
clientes.columns=['id', 'idtransacional', 'nome', 'sexocodigo', 'profissao', 
       'siglanacionalidade', 'nacionalidade',
       'email', 'nascimento', 'cadastro', 'endereco', 'bairro', 'cidade',
       'estado', 'cep', 'pais', 'provedordocliente', 'enderecocompleto',
       'sexodescricao']
clientes=clientes[['id', 'idtransacional', 'nome', 'sexocodigo','sexodescricao',
                   'siglanacionalidade', 'nacionalidade','profissao','email',
                   'provedordocliente','nascimento', 'cadastro','enderecocompleto']]
                   
clientes['id'] += 1
try:
    clientes.to_sql('dim_cliente',if_exists='append',index=False,con=enginedes)
    print("Dimensão cliente carregada com sucesso.")
except exc.SQLAlchemyError as e:
        print(f"Erro ao carregar: {e}")


Dimensão cliente carregada com sucesso.
